# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [10]:
activity=3987156049
LAST=True

In [11]:
import helium
import pandas as pd
import os
import time
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [12]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [13]:
sgmt

,Name,Time,Speed,Power,Info
0,Subida desde la glorieta sur america 1.34km ...,8:59,9.0km/h,229W,"{'Speed': {'Max': '88.2km/h', 'Avg': '4.5km/h'..."
1,Loma Mall Suramerica 0.31km 17m 4%,1:55,9.9km/h,236W,"{'Speed': {'Max': '88.2km/h', 'Avg': '4.5km/h'..."
2,El concejo Prado 3.39km 145m 4%,22:07,9.2km/h,205W,"{'Speed': {'Max': '88.2km/h', 'Avg': '4.5km/h'..."
3,13 % 0.35km 47m 13%,4:28,4.7km/h,211W,"{'Speed': {'Max': '88.2km/h', 'Avg': '4.5km/h'..."
4,Calle 3 Climb 0.51km 49m 9%,4:42,6.5km/h,214W,"{'Speed': {'Max': '88.2km/h', 'Avg': '4.5km/h'..."
5,Carrera 6 Climb 0.40km 23m 6%,2:34,9.5km/h,231W,"{'Speed': {'Max': '88.2km/h', 'Avg': '4.5km/h'..."
6,Remate al Parque SAP 0.59km 35m 5%,4:59,7.1km/h,154W,"{'Speed': {'Max': '88.2km/h', 'Avg': '4.5km/h'..."
7,Ascenso Las playas-Astillero 4.89km 309m 6%,49:20,6.0km/h,167W,"{'Speed': {'Max': '88.2km/h', 'Avg': '4.5km/h'..."
8,Loma de la cascada 0.31km 62m 20%,4:18,4.4km/h,205W,"{'Speed': {'Max': '88.2km/h', 'Avg': '4.5km/h'..."
9,Subida desde El Salado 3.01km 92m 2%,21:28,8.4km/h,166W,"{'Speed': {'Max': '88.2km/h', 'Avg': '4.5km/h'..."


In [14]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [15]:
df.to_json(db_file)

### Save notebook here

In [16]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [17]:
!bash commit.sh

[master d732017] Morning Ride 7:49 AM on Sunday, September 27, 2020
 2 files changed, 308 insertions(+), 912 deletions(-)
 rewrite Strava.ipynb (81%)
 rewrite db/strava.json (79%)
Counting objects: 5, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 14.08 KiB | 0 bytes/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:restrepo/strava.git
   a80bbbb..d732017  master -> master


In [18]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()